<a href="https://colab.research.google.com/github/biranchi2018/BERT-Examples/blob/master/1.Bert-for-TF2%20Toxic%20Comments%20Classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install bert-for-tf2

     |████████████████████████████████| 40kB 2.1MB/s 
  Created wheel for bert-for-tf2: filename=bert_for_tf2-0.14.4-cp36-none-any.whl size=30114 sha256=e94acfcceab463fbf250ea7843d524f93ca9b8c61cb7feabcfef3c68416ba247
  Stored in directory: /root/.cache/pip/wheels/cf/3f/4d/79d7735015a5f523648df90d871ce8e89a7df8185f7703eeab
  Created wheel for py-params: filename=py_params-0.9.7-cp36-none-any.whl size=7302 sha256=36e0ba5c4c88e9d014a541267d56140e6d4c5a84966ff4ceac44533413f32c39
  Stored in directory: /root/.cache/pip/wheels/67/f5/19/b461849a50aefdf4bab47c4756596e82ee2118b8278e5a1980
  Created wheel for params-flow: filename=params_flow-0.8.2-cp36-none-any.whl size=19473 sha256=dd1d7cda4e9b1f23becc4475c55d9db321f28649dab760c28e555107dc395214
  Stored in directory: /root/.cache/pip/wheels/08/c8/7f/81c86b9ff2b86e2c477e3914175be03e679e596067dc630c06
Successfully built bert-for-tf2 py-params params-flow


In [2]:
!pip install sentencepiece

     |████████████████████████████████| 1.1MB 3.3MB/s 


In [0]:
%tensorflow_version 2.x

In [4]:
import tensorflow as tf
import tensorflow_hub as hub
import bert
from tensorflow.keras.models import  Model
from tqdm import tqdm
import numpy as np
from collections import namedtuple
print("TensorFlow Version:",tf.__version__)
print("Hub version: ",hub.__version__)

TensorFlow Version: 2.2.0
Hub version:  0.8.0


In [0]:


bert_layer=hub.KerasLayer("https://tfhub.dev/tensorflow/bert_en_uncased_L-12_H-768_A-12/1",trainable=True)

MAX_SEQ_LEN=128
input_word_ids = tf.keras.layers.Input(shape=(MAX_SEQ_LEN,), dtype=tf.int32,
                                       name="input_word_ids")
input_mask = tf.keras.layers.Input(shape=(MAX_SEQ_LEN,), dtype=tf.int32,
                                   name="input_mask")
segment_ids = tf.keras.layers.Input(shape=(MAX_SEQ_LEN,), dtype=tf.int32,
                                    name="segment_ids")

pooled_output, sequence_output = bert_layer([input_word_ids, input_mask, segment_ids])

In [0]:
x = tf.keras.layers.GlobalAveragePooling1D()(sequence_output)
x = tf.keras.layers.Dropout(0.2)(x)
out = tf.keras.layers.Dense(6, activation="sigmoid", name="dense_output")(x)

model = tf.keras.models.Model(
      inputs=[input_word_ids, input_mask, segment_ids], outputs=out)


model.compile(loss='binary_crossentropy',
                  optimizer='adam',
                  metrics=['accuracy'])


In [7]:
model.summary()

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_word_ids (InputLayer)     [(None, 128)]        0                                            
__________________________________________________________________________________________________
input_mask (InputLayer)         [(None, 128)]        0                                            
__________________________________________________________________________________________________
segment_ids (InputLayer)        [(None, 128)]        0                                            
__________________________________________________________________________________________________
keras_layer (KerasLayer)        [(None, 768), (None, 109482241   input_word_ids[0][0]             
                                                                 input_mask[0][0]             

In [0]:
FullTokenizer=bert.bert_tokenization.FullTokenizer

vocab_file=bert_layer.resolved_object.vocab_file.asset_path.numpy()

do_lower_case=bert_layer.resolved_object.do_lower_case.numpy()

tokenizer=FullTokenizer(vocab_file,do_lower_case)

In [0]:
def get_masks(tokens, max_seq_length):
    return [1]*len(tokens) + [0] * (max_seq_length - len(tokens))

def get_segments(tokens, max_seq_length):
    """Segments: 0 for the first sequence, 1 for the second"""
    segments = []
    current_segment_id = 0
    for token in tokens:
        segments.append(current_segment_id)
        if token == "[SEP]":
            current_segment_id = 1
    return segments + [0] * (max_seq_length - len(tokens))

In [0]:
def get_ids(tokens, tokenizer, max_seq_length):
    """Token ids from Tokenizer vocab"""
    token_ids = tokenizer.convert_tokens_to_ids(tokens,)
    input_ids = token_ids + [0] * (max_seq_length-len(token_ids))
    return input_ids

In [0]:
import os
os.environ['KAGGLE_USERNAME'] = "username" # username from the json file
os.environ['KAGGLE_KEY'] = "kagglekey" # key from the json file

In [13]:
!kaggle competitions download -c jigsaw-toxic-comment-classification-challenge

  0% 0.00/1.39M [00:00<?, ?B/s]
100% 1.39M/1.39M [00:00<00:00, 97.6MB/s]
 98% 23.0M/23.4M [00:01<00:00, 14.2MB/s]
100% 23.4M/23.4M [00:01<00:00, 14.7MB/s]
 68% 18.0M/26.3M [00:00<00:00, 27.5MB/s]
100% 26.3M/26.3M [00:00<00:00, 31.9MB/s]
  0% 0.00/1.46M [00:00<?, ?B/s]
100% 1.46M/1.46M [00:00<00:00, 199MB/s]


In [14]:
!unzip train.csv.zip
!unzip test.csv.zip	


Archive:  train.csv.zip
  inflating: train.csv               
Archive:  test.csv.zip
  inflating: test.csv                


In [15]:
!ls

sample_data		   test.csv	 test_labels.csv.zip  train.csv.zip
sample_submission.csv.zip  test.csv.zip  train.csv


In [0]:
import pandas as pd

df=pd.read_csv('train.csv')

df = df.sample(frac=1)


In [0]:
train_sentences = df["comment_text"].fillna("CVxTz").values
list_classes = ["toxic", "severe_toxic", "obscene", "threat", "insult", "identity_hate"]
train_y = df[list_classes].values

In [18]:
train_sentences[:2]

array(["Someone should move the quotes to wikiquote, I would but I don't know how.",
       'Why is linking the province any different if it is beside a city? I still do not see the crime? The user when clicking on the link can see the underline to see if it is on the one link or separate. Also, why have you fixed only Flo Rida, but not the others?'],
      dtype=object)

In [19]:
train_y[:2]

array([[0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0]])

In [0]:
def create_single_input(sentence,MAX_LEN):
  
  stokens = tokenizer.tokenize(sentence)
  
  stokens = stokens[:MAX_LEN]
  
  stokens = ["[CLS]"] + stokens + ["[SEP]"]
 
  ids = get_ids(stokens, tokenizer, MAX_SEQ_LEN)
  masks = get_masks(stokens, MAX_SEQ_LEN)
  segments = get_segments(stokens, MAX_SEQ_LEN)

  return ids,masks,segments

In [0]:
def create_input_array(sentences):

  input_ids, input_masks, input_segments = [], [], []

  for sentence in tqdm(sentences,position=0, leave=True):
  
    ids,masks,segments=create_single_input(sentence,MAX_SEQ_LEN-2)

    input_ids.append(ids)
    input_masks.append(masks)
    input_segments.append(segments)

  return [np.asarray(input_ids, dtype=np.int32), 
            np.asarray(input_masks, dtype=np.int32), 
            np.asarray(input_segments, dtype=np.int32)]

In [22]:
inputs=create_input_array(train_sentences)

100%|██████████| 159571/159571 [03:02<00:00, 876.27it/s]


In [23]:
inputs[:2]

[array([[  101,  2619,  2323, ...,     0,     0,     0],
        [  101,  2339,  2003, ...,     0,     0,     0],
        [  101,  2562,  2033, ...,     0,     0,     0],
        ...,
        [  101, 23561,  1024, ...,     0,     0,     0],
        [  101,  1000,  1010, ...,     0,     0,     0],
        [  101,  1000,  1024, ...,  2011,  9530,   102]], dtype=int32),
 array([[1, 1, 1, ..., 0, 0, 0],
        [1, 1, 1, ..., 0, 0, 0],
        [1, 1, 1, ..., 0, 0, 0],
        ...,
        [1, 1, 1, ..., 0, 0, 0],
        [1, 1, 1, ..., 0, 0, 0],
        [1, 1, 1, ..., 1, 1, 1]], dtype=int32)]

In [24]:
model.fit(inputs,train_y,epochs=1,batch_size=32,validation_split=0.2,shuffle=True)

2152/3990 [===============>..............] - ETA: 2:20:11 - loss: 0.1465 - accuracy: 0.8538

KeyboardInterrupt: ignored

In [25]:
test_df=pd.read_csv("test.csv")

test_sentences = test_df["comment_text"].fillna("CVxTz").values

test_sentences[110:150]


array(["== Second issue == \n\n I'm thinking we need a taxonomy section; there are things that aren't really as clear as they maybe could be. Why is/what specific findings about the fossils places R. narmadensis in the subfamily Carnotaurinae? Why 'princely lizard of the Narmada'; is that significant as to the stature/behavior/predatorial prowess of R. narmadensis? If there is anything more precise on its taxonomy, such as that, it would be great to add.",
       '" \n\n == Hey man == \n\n  \n I saw you removing all the fair use images from Jimi Hendrix discography. It\'s a bitch to do it, but good job doing the right thing. Right now I\'m going through a lot of artist and discography pages looking for album covers to get rid of and other bad images. You got my back. Keep up the good work.++++ "',
       ": But there's only one EP and one album, which is really also an EP. Even in the 70s, it would have been considered an EP, although it could have been considered an album by length in

In [26]:
test_inputs=create_input_array(test_sentences[110:150])

print(model.predict(test_inputs))

100%|██████████| 40/40 [00:00<00:00, 648.80it/s]


[[0.09538716 0.00635716 0.05412841 0.00132161 0.04550296 0.01215178]
 [0.0953871  0.00635716 0.05412841 0.00132161 0.04550296 0.01215178]
 [0.09538704 0.00635716 0.05412844 0.00132161 0.04550296 0.01215178]
 [0.09538713 0.00635719 0.05412841 0.00132161 0.04550296 0.01215178]
 [0.0953871  0.00635716 0.05412835 0.00132161 0.04550296 0.01215172]
 [0.09538716 0.00635716 0.05412841 0.00132161 0.04550296 0.01215178]
 [0.09538716 0.00635716 0.05412832 0.00132161 0.04550308 0.01215181]
 [0.09538716 0.00635716 0.05412841 0.00132161 0.04550296 0.01215178]
 [0.0953871  0.00635716 0.05412844 0.00132161 0.04550296 0.01215178]
 [0.0953871  0.00635716 0.05412841 0.00132161 0.04550296 0.01215172]
 [0.09538713 0.00635716 0.05412835 0.00132161 0.04550296 0.01215178]
 [0.09538701 0.00635716 0.05412841 0.00132161 0.04550302 0.01215172]
 [0.0953871  0.00635716 0.05412844 0.00132161 0.04550296 0.01215178]
 [0.09538698 0.00635716 0.05412841 0.00132161 0.0455029  0.01215172]
 [0.09538713 0.00635716 0.05412841